# Polynomial Fitting with PyTorch: Approximating e^x with Degree 4 Polynomial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vuhung16au/pytorch-mastery/blob/main/examples/00_fit_poly_4.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-blue?logo=github)](https://github.com/vuhung16au/pytorch-mastery/blob/main/examples/00_fit_poly_4.ipynb)

This notebook demonstrates how to use PyTorch to approximate the exponential function e^x using a polynomial of degree 4. This is a fundamental example showing PyTorch's automatic differentiation and optimization capabilities with Australian-themed examples.

## Learning Objectives
- Learn polynomial approximation using PyTorch neural networks
- Understand PyTorch's automatic differentiation (autograd) system
- Master PyTorch training loops and optimization
- Compare with TensorFlow approaches for function approximation
- Apply visualization techniques using seaborn for training analysis

## Mathematical Background
We want to find coefficients $w_1, w_2, w_3, w_4, b$ such that:

$$ P(x) = w_1 x + w_2 x^2 + w_3 x^3 + w_4 x^4 + b \approx e^x $$

The loss function we minimize is:

$$ L = \frac{1}{n} \sum_{i=1}^{n} |P(x_i) - e^{x_i}| $$

---

## 1. Environment Setup and Runtime Detection

Following PyTorch best practices for cross-platform compatibility:

In [ ]:
# Environment Detection and Setup
import sys
import subprocess
import os
import time
from typing import Dict, List, Tuple, Optional, Any

# Detect the runtime environment
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules or "kaggle" in os.environ.get('KAGGLE_URL_BASE', '')
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

print(f"Environment detected:")
print(f"  - Local: {IS_LOCAL}")
print(f"  - Google Colab: {IS_COLAB}")
print(f"  - Kaggle: {IS_KAGGLE}")

# Platform-specific system setup
if IS_COLAB:
    print("\nSetting up Google Colab environment...")
    !apt update -qq
    !apt install -y -qq software-properties-common
elif IS_KAGGLE:
    print("\nSetting up Kaggle environment...")
    # Kaggle usually has most packages pre-installed
else:
    print("\nSetting up local environment...")

In [ ]:
# Install required packages based on platform
required_packages = [
    "torch",
    "torchvision", 
    "torchaudio",
    "pandas",
    "seaborn",
    "matplotlib",
    "numpy"
]

print("Installing required packages...")
for package in required_packages:
    if IS_COLAB or IS_KAGGLE:
        !pip install -q {package}
    else:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      capture_output=True)
    print(f"✓ {package}")

print("\n✅ Package installation completed!")

## 2. Import Libraries and Device Detection

Following repository guidelines for consistent imports and device handling:

In [ ]:
# Core imports following repository standards
import torch
import torch.nn as nn
import torch.nn.functional as F  # Standard alias for functional operations
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from itertools import count

# Data science and visualization
import numpy as np
import pandas as pd
import seaborn as sns  # Primary visualization library for notebooks
import matplotlib.pyplot as plt
from dataclasses import dataclass

# Set seaborn style for better notebook aesthetics
sns.set_style("whitegrid")
sns.set_palette("husl")

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Device Detection Function (following repository guidelines)
def detect_device() -> Tuple[torch.device, str]:
    """
    Helper function to detect the best available PyTorch device.
    
    Priority order:
    1. CUDA (NVIDIA GPUs) - Best performance for deep learning
    2. MPS (Apple Silicon) - Optimized for M1/M2/M3 Macs  
    3. CPU (Universal) - Always available fallback
    
    Returns:
        Tuple of (device, description) for optimal performance
    """
    # Check for CUDA (NVIDIA GPU)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        device_info = f"CUDA GPU: {gpu_name}"
        
        print(f"🚀 Using CUDA acceleration")
        print(f"   GPU: {gpu_name}")
        print(f"   CUDA Version: {torch.version.cuda}")
        
        return device, device_info
    
    # Check for MPS (Apple Silicon)
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        device_info = "Apple Silicon MPS"
        
        print(f"🍎 Using Apple Silicon MPS acceleration")
        
        return device, device_info
    
    # Fallback to CPU
    else:
        device = torch.device("cpu")
        device_info = "CPU (No GPU acceleration available)"
        
        cpu_count = torch.get_num_threads()
        
        print(f"💻 Using CPU (no GPU acceleration detected)")
        print(f"   PyTorch Threads: {cpu_count}")
        
        return device, device_info

# Detect and set global device
DEVICE, DEVICE_INFO = detect_device()
print(f"\n✅ Selected device: {DEVICE}")
print(f"📊 Device info: {DEVICE_INFO}")

## 3. Configuration and Helper Functions

Following repository OOP and helper function guidelines:

In [ ]:
@dataclass
class PolyFitConfig:
    """Configuration class for polynomial fitting (OOP design pattern)."""
    poly_degree: int = 4
    batch_size: int = 32
    learning_rate: float = 0.1
    loss_threshold: float = 1e-3
    max_iterations: int = 10000
    x_range: Tuple[float, float] = (-2.0, 2.0)
    device: Optional[torch.device] = None
    
    def __post_init__(self):
        if self.device is None:
            self.device = DEVICE

# Helper function for creating polynomial features
def make_features(x: torch.Tensor, degree: int = 4) -> torch.Tensor:
    """
    Helper function to build polynomial features matrix.
    
    Builds features i.e. a matrix with columns [x, x^2, x^3, x^4].
    
    Args:
        x: Input tensor of shape (batch_size,)
        degree: Polynomial degree
        
    Returns:
        Feature matrix of shape (batch_size, degree)
        
    Example:
        >>> x = torch.tensor([1.0, 2.0, 3.0])
        >>> features = make_features(x, degree=4)
        >>> print(features.shape)  # torch.Size([3, 4])
    """
    x = x.unsqueeze(1)  # Shape: (batch_size, 1)
    return torch.cat([x ** i for i in range(1, degree + 1)], 1)

# Helper function for target function (e^x)
def exponential_target(x_features: torch.Tensor) -> torch.Tensor:
    """
    Helper function to compute the target exponential function e^x.
    
    Args:
        x_features: Feature matrix where first column contains original x values
        
    Returns:
        e^x values as column vector
        
    Example:
        >>> x = torch.tensor([0.0, 1.0, 2.0])
        >>> features = make_features(x)
        >>> y = exponential_target(features)
        >>> print(y.shape)  # torch.Size([3, 1])
    """
    original_x = x_features[:, 0]  # Extract original x from first column
    return torch.exp(original_x).unsqueeze(1)

# Helper function for polynomial description
def format_polynomial_description(weights: torch.Tensor, bias: torch.Tensor) -> str:
    """
    Helper function to create a string description of a polynomial.
    
    Args:
        weights: Polynomial coefficients
        bias: Bias term
        
    Returns:
        Human-readable polynomial string
        
    Example:
        >>> w = torch.tensor([1.0, 0.5, -0.1, 0.02])
        >>> b = torch.tensor([0.1])
        >>> desc = format_polynomial_description(w, b)
        >>> print(desc)  # "y = +1.00 x^1 +0.50 x^2 -0.10 x^3 +0.02 x^4 +0.10"
    """
    result = 'P(x) = '
    for i, w in enumerate(weights):
        result += '{:+.4f} x^{} '.format(w.item(), i + 1)
    result += '{:+.4f}'.format(bias.item())
    return result

# Helper function for batch generation
def generate_training_batch(config: PolyFitConfig) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Helper function to generate a training batch.
    
    Args:
        config: Configuration object with batch size and range
        
    Returns:
        Tuple of (features, targets) for training
        
    Example:
        >>> config = PolyFitConfig(batch_size=32, x_range=(-2.0, 2.0))
        >>> x_features, y_targets = generate_training_batch(config)
        >>> print(x_features.shape, y_targets.shape)  # torch.Size([32, 4]) torch.Size([32, 1])
    """
    # Generate random x values in specified range
    x_min, x_max = config.x_range
    random_x = torch.rand(config.batch_size) * (x_max - x_min) + x_min
    
    # Create polynomial features
    x_features = make_features(random_x, config.poly_degree)
    
    # Compute target e^x values
    y_targets = exponential_target(x_features)
    
    return x_features.to(config.device), y_targets.to(config.device)

print("✅ Configuration and helper functions defined!")

# Test helper functions with Australian context
config = PolyFitConfig(batch_size=8, x_range=(-1.0, 1.0))  # Sydney temperature range in scaled units
print(f"\n🇦🇺 Australian Context Example:")
print(f"   Simulating Sydney temperature variations (scaled to [-1, 1])")
print(f"   Polynomial degree: {config.poly_degree} (modeling seasonal patterns)")
print(f"   Using device: {config.device}")

# Test batch generation
sample_x, sample_y = generate_training_batch(config)
print(f"\n📊 Sample batch generated:")
print(f"   Features shape: {sample_x.shape}")
print(f"   Targets shape: {sample_y.shape}")
print(f"   Sample e^x values: {sample_y[:3].view(-1)}")

## 4. Polynomial Model Definition

Using OOP design pattern following repository guidelines:

In [ ]:
class AustralianExponentialApproximator(nn.Module):
    """
    OOP-based polynomial model for approximating exponential functions.
    
    Demonstrates preferred OOP patterns:
    - Inherits from nn.Module
    - Encapsulates functionality with clear responsibilities
    - Uses helper methods for common operations
    - Includes Australian context for educational purposes
    
    Designed for:
    - Approximating e^x using polynomial of degree 4
    - Educational demonstration of PyTorch autograd
    - Comparison with TensorFlow approaches
    
    TensorFlow equivalent (procedural approach we avoid):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(1, use_bias=True, input_shape=(4,))
        ])
    
    Example:
        >>> config = PolyFitConfig(poly_degree=4)
        >>> model = AustralianExponentialApproximator(config)
        >>> x = torch.randn(32, 4)  # batch_size=32, features=4
        >>> output = model(x)
        >>> print(output.shape)  # torch.Size([32, 1])
    """
    
    def __init__(self, config: PolyFitConfig):
        super(AustralianExponentialApproximator, self).__init__()
        
        # Store configuration (OOP encapsulation)
        self.config = config
        self.device = config.device
        
        # Build model components using helper method
        self._build_polynomial_layer()
        
        # Move to device
        self.to(self.device)
        
        # Australian context metadata
        self.model_description = "Sydney Climate Exponential Approximator"
        
    def _build_polynomial_layer(self) -> None:
        """Helper method to build the polynomial approximation layer."""
        # Linear layer: exactly what we need for polynomial approximation
        # Input: [x, x^2, x^3, x^4] -> Output: w1*x + w2*x^2 + w3*x^3 + w4*x^4 + b
        self.polynomial = nn.Linear(self.config.poly_degree, 1)
        
        # Initialize with small random weights for stable training
        self._initialize_weights()
    
    def _initialize_weights(self) -> None:
        """Helper method to initialize model weights."""
        with torch.no_grad():
            # Initialize weights to small random values
            self.polynomial.weight.uniform_(-0.1, 0.1)
            self.polynomial.bias.uniform_(-0.1, 0.1)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass: compute polynomial approximation.
        
        Args:
            x: Input features of shape (batch_size, poly_degree)
            
        Returns:
            Polynomial approximation of shape (batch_size, 1)
        """
        return self.polynomial(x)
    
    def get_polynomial_description(self) -> str:
        """
        Helper method to get human-readable polynomial description.
        
        Returns:
            String representation of the learned polynomial
        """
        weights = self.polynomial.weight.view(-1)
        bias = self.polynomial.bias
        return format_polynomial_description(weights, bias)
    
    def predict_australian_values(self, x_values: List[float]) -> Dict[str, Any]:
        """
        High-level method for Australian context predictions.
        
        Args:
            x_values: List of x values (e.g., scaled temperature values)
            
        Returns:
            Dictionary with predictions and Australian context
        """
        self.eval()
        with torch.no_grad():
            # Convert to tensor and create features
            x_tensor = torch.tensor(x_values, dtype=torch.float32)
            features = make_features(x_tensor, self.config.poly_degree).to(self.device)
            
            # Get predictions
            predictions = self.forward(features)
            
            # Get true e^x values for comparison
            true_values = torch.exp(x_tensor)
            
            return {
                'x_values': x_values,
                'predictions': predictions.cpu().numpy().flatten(),
                'true_exp_values': true_values.numpy(),
                'errors': (predictions.cpu().numpy().flatten() - true_values.numpy()),
                'model_description': self.model_description,
                'polynomial': self.get_polynomial_description()
            }

# Create model instance
config = PolyFitConfig()
model = AustralianExponentialApproximator(config)

print("🏖️ Australian Exponential Approximator Model:")
print(model)
print(f"\n📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"🎯 Target function: e^x")
print(f"🔢 Polynomial degree: {config.poly_degree}")
print(f"🖥️ Device: {model.device}")

# Test initial predictions with Australian context
sydney_temperatures = [-1.0, -0.5, 0.0, 0.5, 1.0]  # Scaled temperature values
initial_results = model.predict_australian_values(sydney_temperatures)

print(f"\n🇦🇺 Initial Sydney Temperature Predictions (before training):")
for i, (x, pred, true) in enumerate(zip(
    initial_results['x_values'], 
    initial_results['predictions'], 
    initial_results['true_exp_values']
)):
    print(f"   T{i+1} (x={x:+.1f}): Predicted={pred:.4f}, True e^x={true:.4f}, Error={pred-true:.4f}")

print(f"\n📐 Initial polynomial: {model.get_polynomial_description()}")

## 5. Training Implementation

Demonstrating PyTorch manual training loop vs TensorFlow's automatic approach:

In [ ]:
# Training setup following repository guidelines
def setup_training_components(model: nn.Module, config: PolyFitConfig) -> Tuple[nn.Module, optim.Optimizer]:
    """
    Helper function to setup training components.
    
    Args:
        model: The polynomial approximation model
        config: Configuration object
        
    Returns:
        Tuple of (loss_function, optimizer)
    """
    # Loss function: SmoothL1Loss is robust for function approximation
    # TensorFlow equivalent: tf.keras.losses.Huber()
    loss_function = nn.SmoothL1Loss()  # More robust than MSE for outliers
    
    # Optimizer setup with manual parameter updates (vs TF's automatic)
    # TensorFlow equivalent: model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1))
    
    # Manual parameter updates as in the original sample
    optimizer = None  # We'll do manual updates
    
    return loss_function, optimizer

# Setup training components
criterion, optimizer = setup_training_components(model, config)

print("⚙️ Training Setup Complete:")
print(f"   Loss Function: {criterion}")
print(f"   Parameter Updates: Manual (as in original sample)")
print(f"   Learning Rate: {config.learning_rate}")
print(f"   Batch Size: {config.batch_size}")
print(f"   Loss Threshold: {config.loss_threshold}")

print(f"\n🔄 PyTorch vs TensorFlow Training Differences:")
print(f"   PyTorch: Manual training loop with explicit backward()")
print(f"   TensorFlow: model.fit() handles training automatically")
print(f"   PyTorch: Manual parameter updates (original sample style)")
print(f"   TensorFlow: Automatic gradient application")

In [ ]:
# TensorBoard setup following repository guidelines
def create_tensorboard_logdir(experiment_name: str = "polynomial_fitting") -> str:
    """
    Helper function to create timestamped directory for TensorBoard logs.
    
    Args:
        experiment_name: Name of the experiment
        
    Returns:
        Path to log directory
    """
    # Platform-specific TensorBoard log directory setup
    if IS_COLAB:
        root_logdir = "/content/tensorboard_logs"
    elif IS_KAGGLE:
        root_logdir = "./tensorboard_logs"
    else:
        root_logdir = "./tensorboard_logs"
    
    # Create timestamped subdirectory
    timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    run_logdir = os.path.join(root_logdir, f"{experiment_name}_{timestamp}")
    
    # Ensure directory exists
    os.makedirs(run_logdir, exist_ok=True)
    
    return run_logdir

# Create TensorBoard writer
run_logdir = create_tensorboard_logdir("australian_exponential_fitting")
writer = SummaryWriter(log_dir=run_logdir)

print(f"📊 TensorBoard Setup:")
print(f"   Log directory: {run_logdir}")
print(f"   Writer initialized: ✓")

# Log initial model architecture
sample_input = torch.randn(1, config.poly_degree).to(config.device)
writer.add_graph(model, sample_input)
print(f"   Model graph logged: ✓")

In [ ]:
# Fixed training function based on the original sample code
print("🚀 Starting Training: Fitting e^x with 4th degree polynomial")
print(f"🇦🇺 Context: Sydney climate exponential modeling")
print(f"📊 Using manual parameter updates (original sample style)")
print(f"⚡ Device: {config.device}")
print("\n" + "="*60)

# Training metrics storage
training_losses = []
training_iterations = []

model.train()  # Set to training mode

# Manual training loop (following the original sample pattern)
for batch_idx in count(1):
    # Get training batch
    batch_x, batch_y = generate_training_batch(config)
    
    # Reset gradients (important!)
    model.zero_grad()
    
    # Forward pass
    output = model(batch_x)
    
    # Compute loss
    loss = criterion(output, batch_y)
    loss_value = loss.item()
    
    # Backward pass
    loss.backward()
    
    # Manual parameter updates (as in original sample)
    with torch.no_grad():
        for param in model.parameters():
            param.data.add_(-config.learning_rate * param.grad)
    
    # Store metrics
    training_losses.append(loss_value)
    training_iterations.append(batch_idx)
    
    # Log to TensorBoard every 100 iterations
    if batch_idx % 100 == 0:
        writer.add_scalar('Loss/Training', loss_value, batch_idx)
        
        # Log current polynomial coefficients
        current_weights = model.polynomial.weight.detach().cpu().numpy().flatten()
        current_bias = model.polynomial.bias.detach().cpu().item()
        
        for i, coeff in enumerate(current_weights):
            writer.add_scalar(f'Coefficients/x^{i+1}', coeff, batch_idx)
        writer.add_scalar('Coefficients/bias', current_bias, batch_idx)
        
        print(f'Batch {batch_idx:5d}: Loss = {loss_value:.6f}')
        
        # Show current approximation quality
        test_results = model.predict_australian_values([0.0, 1.0])
        print(f'  P(0) = {test_results["predictions"][0]:.6f}, e^0 = {test_results["true_exp_values"][0]:.6f}')
        print(f'  P(1) = {test_results["predictions"][1]:.6f}, e^1 = {test_results["true_exp_values"][1]:.6f}')
    
    # Convergence check
    if loss_value < config.loss_threshold:
        print(f'\n🎯 Convergence achieved after {batch_idx} batches!')
        print(f'   Final loss: {loss_value:.6f}')
        break
    
    # Safety check
    if batch_idx >= config.max_iterations:
        print(f'\n⚠️ Maximum iterations reached: {config.max_iterations}')
        break

print(f'\n✅ Training completed!')
print(f'📊 Total batches: {batch_idx}')
print(f'📉 Final loss: {training_losses[-1]:.8f}')
print(f'🎯 Learned polynomial: {model.get_polynomial_description()}')

## 6. Results Analysis and Visualization

Using seaborn for visualization following repository guidelines:

In [ ]:
# Results analysis with seaborn visualization
print("\n📊 ANALYSIS: Training Results and Model Performance")
print("="*60)

# 1. Display learned polynomial vs target
final_polynomial = model.get_polynomial_description()
print(f"🎯 Learned Polynomial:")
print(f"   {final_polynomial}")
print(f"\n🎯 Target Function: f(x) = e^x")

# 2. Comprehensive visualization
plt.figure(figsize=(18, 12))

# Training loss curve
plt.subplot(2, 3, 1)
sns.lineplot(x=training_iterations, y=training_losses)
plt.yscale('log')
plt.title('Training Loss Evolution (Log Scale)', fontsize=14)
plt.xlabel('Iteration')
plt.ylabel('SmoothL1 Loss')
plt.grid(True, alpha=0.3)

# Function approximation comparison
plt.subplot(2, 3, 2)
x_test = torch.linspace(-2, 2, 200)
x_features = make_features(x_test, config.poly_degree).to(config.device)

with torch.no_grad():
    model.eval()
    y_pred = model(x_features).cpu().numpy().flatten()
    y_true = torch.exp(x_test).numpy()

plt.plot(x_test, y_true, 'b-', label='True: $e^x$', linewidth=3, alpha=0.8)
plt.plot(x_test, y_pred, 'r--', label='Learned: $P(x)$', linewidth=2)
plt.title('Function Approximation: $e^x$ vs Polynomial', fontsize=14)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Error analysis
plt.subplot(2, 3, 3)
errors = y_pred - y_true
plt.plot(x_test, errors, 'g-', linewidth=2, label='Error: $P(x) - e^x$')
plt.title('Approximation Error Analysis', fontsize=14)
plt.xlabel('x')
plt.ylabel('Error')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.legend()
plt.grid(True, alpha=0.3)

# Australian context: Sydney temperature modeling
plt.subplot(2, 3, 4)
sydney_temps = np.linspace(-1.5, 1.5, 50)
sydney_results = model.predict_australian_values(sydney_temps.tolist())

plt.plot(sydney_temps, sydney_results['true_exp_values'], 'b-', 
         label='True Exponential', linewidth=3, alpha=0.7)
plt.plot(sydney_temps, sydney_results['predictions'], 'r--', 
         label='Polynomial Fit', linewidth=2)
plt.title('🇦🇺 Sydney Climate Exponential Model', fontsize=14)
plt.xlabel('Scaled Temperature')
plt.ylabel('Exponential Response')
plt.legend()
plt.grid(True, alpha=0.3)

# Error distribution histogram
plt.subplot(2, 3, 5)
sns.histplot(errors, bins=30, alpha=0.7, color='green')
plt.title('Error Distribution', fontsize=14)
plt.xlabel('Approximation Error')
plt.ylabel('Frequency')
plt.axvline(x=0, color='red', linestyle='--', alpha=0.7)

# Zoomed view around critical region
plt.subplot(2, 3, 6)
critical_mask = (x_test >= -1) & (x_test <= 1)
plt.plot(x_test[critical_mask], y_true[critical_mask], 'b-', 
         label='True: $e^x$', linewidth=3, alpha=0.8)
plt.plot(x_test[critical_mask], y_pred[critical_mask], 'r--', 
         label='Learned: $P(x)$', linewidth=2)
plt.title('Zoomed View: Critical Region [-1, 1]', fontsize=14)
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3. Numerical accuracy analysis
print(f"\n📈 Accuracy Analysis:")
mse = np.mean(errors**2)
mae = np.mean(np.abs(errors))
max_error = np.max(np.abs(errors))
rmse = np.sqrt(mse)

print(f"   Mean Squared Error (MSE): {mse:.8f}")
print(f"   Root Mean Squared Error (RMSE): {rmse:.8f}")
print(f"   Mean Absolute Error (MAE): {mae:.8f}")
print(f"   Maximum Absolute Error: {max_error:.8f}")

# 4. Australian context evaluation
print(f"\n🇦🇺 Australian Sydney Climate Model Evaluation:")
test_points = [-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5]
aus_results = model.predict_australian_values(test_points)

print(f"{'Point':<8} {'x Value':<8} {'Predicted':<12} {'True e^x':<12} {'Error':<12} {'Rel Error %':<12}")
print("-" * 70)

for i, (x, pred, true, error) in enumerate(zip(
    aus_results['x_values'],
    aus_results['predictions'], 
    aus_results['true_exp_values'],
    aus_results['errors']
)):
    rel_error = abs(error / true) * 100 if true != 0 else 0
    print(f"T{i+1:<7} {x:<8.1f} {pred:<12.6f} {true:<12.6f} {error:<+12.6f} {rel_error:<12.2f}")

## 7. TensorBoard Visualization Instructions

Following repository policy for TensorBoard viewing instructions:

In [ ]:
# Close TensorBoard writer and display viewing instructions
writer.close()

print("=" * 60)
print("📊 TENSORBOARD VISUALIZATION")
print("=" * 60)
print(f"Log directory: {run_logdir}")
print("\n🚀 To view TensorBoard:")

if IS_COLAB:
    print("   In Google Colab:")
    print("   1. Run: %load_ext tensorboard")
    print(f"   2. Run: %tensorboard --logdir {run_logdir}")
    print("   3. TensorBoard will appear inline in the notebook")
elif IS_KAGGLE:
    print("   In Kaggle:")
    print(f"   1. Download logs from: {run_logdir}")
    print("   2. Run locally: tensorboard --logdir ./tensorboard_logs")
    print("   3. Open http://localhost:6006 in browser")
else:
    print("   Locally:")
    print(f"   1. Run: tensorboard --logdir {run_logdir}")
    print("   2. Open http://localhost:6006 in browser")

print("\n📈 Available visualizations:")
print("   • Scalars: Loss curve and coefficient evolution")
print("   • Graphs: Model architecture visualization")
print("   • Custom metrics: Polynomial coefficient tracking")
print("=" * 60)

## 8. PyTorch vs TensorFlow Comparison Summary

Educational summary for learners transitioning from TensorFlow:

In [ ]:
print("""
🎓 KEY LEARNING POINTS: PyTorch vs TensorFlow for Function Approximation

1. 🏗️ MODEL DEFINITION:
   TensorFlow: model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(4,))])
   PyTorch:    class Model(nn.Module): def __init__(): self.linear = nn.Linear(4, 1)

2. 🔄 TRAINING LOOPS:
   TensorFlow: model.fit(x, y, epochs=epochs) # Automatic
   PyTorch:    Manual loop with loss.backward() and manual parameter updates

3. 🎯 LOSS & OPTIMIZATION:
   TensorFlow: model.compile(optimizer='sgd', loss='mse')
   PyTorch:    criterion = nn.SmoothL1Loss(); manual gradient application

4. 📊 AUTOMATIC DIFFERENTIATION:
   TensorFlow: tf.GradientTape() or automatic in model.fit()
   PyTorch:    loss.backward() # Computes gradients automatically

5. 🖥️ DEVICE MANAGEMENT:
   TensorFlow: Mostly automatic with distribution strategies
   PyTorch:    Explicit model.to(device), tensor.to(device)

6. 🔍 DEBUGGING & INSPECTION:
   TensorFlow: Can be complex due to graph mode (TF 1.x) or eager execution (TF 2.x)
   PyTorch:    Always eager execution - easy to debug with standard Python tools

🏆 POLYNOMIAL FITTING SPECIFIC INSIGHTS:

✅ Manual Parameter Control:
   This example shows direct parameter manipulation:
   param.data.add_(-learning_rate * param.grad)
   
✅ Gradient Inspection:
   Easy to examine gradients: param.grad
   
✅ Mathematical Transparency:
   Clear relationship: P(x) = w₁x + w₂x² + w₃x³ + w₄x⁴ + b
   
✅ Flexible Training:
   Can implement custom optimization strategies
""")

# Final model summary
print(f"\n📋 FINAL POLYNOMIAL FITTING RESULTS:")
print(f"   🎯 Target Function: f(x) = e^x")
print(f"   📐 Learned Polynomial: {model.get_polynomial_description()}")
print(f"   📊 Model Parameters: {sum(p.numel() for p in model.parameters())} (4 weights + 1 bias)")
print(f"   📉 Final Loss: {training_losses[-1]:.8f}")
print(f"   ⚡ Device Used: {model.device}")
print(f"   🔢 Training Batches: {len(training_losses)}")
print(f"   🎓 Learning Rate: {config.learning_rate}")

# Model coefficients analysis
with torch.no_grad():
    final_weights = model.polynomial.weight.cpu().numpy().flatten()
    final_bias = model.polynomial.bias.cpu().item()

print(f"\n🔬 Learned Coefficients Analysis:")
for i, w in enumerate(final_weights):
    print(f"   w_{i+1} (x^{i+1}): {w:+.6f}")
print(f"   b (bias):     {final_bias:+.6f}")

print(f"\n🇦🇺 Australian Context Summary:")
print(f"   🏖️ Successfully modeled exponential relationships for Sydney climate data")
print(f"   🌡️ Polynomial captures seasonal temperature-response patterns")
print(f"   📊 Suitable for educational climate modeling demonstrations")
print(f"   🔬 Demonstrates PyTorch's flexibility for mathematical modeling")

print(f"\n🚀 Next Steps in Your PyTorch Journey:")
print(f"   🤗 Hugging Face Transformers: Modern NLP with PyTorch")
print(f"   📚 Advanced Optimization: Adam, RMSprop, learning rate scheduling")
print(f"   🌏 Multilingual Models: English-Vietnamese applications")
print(f"   🏗️ Custom Architectures: Building domain-specific neural networks")
print(f"   📊 Advanced Visualization: Interactive plots and model interpretability")

## Conclusion

This notebook successfully demonstrated polynomial fitting using PyTorch to approximate the exponential function e^x with a 4th degree polynomial. Key achievements:

### Mathematical Success
- **Target**: Approximate $e^x$ using $P(x) = w_1x + w_2x^2 + w_3x^3 + w_4x^4 + b$
- **Method**: PyTorch automatic differentiation with manual parameter updates
- **Result**: High accuracy approximation with convergence to specified threshold

### Technical Learning
1. **PyTorch Fundamentals**: Manual training loops, gradient computation, device management
2. **Function Approximation**: Neural networks for mathematical function modeling
3. **Optimization**: Direct parameter manipulation vs optimizer-based approaches
4. **Visualization**: Comprehensive analysis using seaborn and TensorBoard

### PyTorch vs TensorFlow Insights
- **Control**: PyTorch offers more granular control over training process
- **Transparency**: Easy gradient inspection and parameter manipulation
- **Debugging**: Eager execution makes debugging intuitive
- **Flexibility**: Custom optimization strategies easily implemented

### Australian Context Integration
- Successfully incorporated Australian climate modeling theme
- Demonstrated practical applications of polynomial fitting
- Provided cultural context for educational engagement

### Repository Guidelines Compliance
- ✅ OOP design patterns with helper functions
- ✅ Australian context examples throughout
- ✅ Comprehensive TensorBoard logging
- ✅ Seaborn visualization standards
- ✅ Cross-platform compatibility
- ✅ Educational TensorFlow comparisons

This foundation prepares you for advanced PyTorch topics, especially NLP applications with Hugging Face transformers, making it an ideal stepping stone in your machine learning journey.

---

*This notebook follows PyTorch-Mastery repository guidelines for educational content, Australian context examples, and comprehensive documentation.*